In [1]:
import pickle
from keras.src.models.model import model_from_json
from sklearn.preprocessing import LabelEncoder
from matplotlib import pyplot as plt
import numpy as np
from matplotlib.colors import ListedColormap

In [21]:
model_path_directory = '../../AIModels/BaselineModel/'
step = 0.02
lateral_min = -2.5
lateral_max = 2.5
longitudinal_min = -2.5
longitudinal_max = 2.5

custom_color_map = { 
    'Acceptable': 'yellow',
    'Excellent': 'green',
    'So and So': 'orange',
    'Uncomfortable': 'red'
}

class_names = ['Acceptable','Excellent','So and So','Uncomfortable']

custom_listed_color_map = ListedColormap([custom_color_map[label] for label in class_names])

In [22]:
with open(model_path_directory + 'DrivingComfortabilityPredictingModel.json', 'r') as json_file:
    model_json = json_file.read()
loaded_baseline_model = model_from_json(model_json)

loaded_baseline_model.load_weights(model_path_directory + 'DrivingComfortabilityPredictingModel.weights.h5')

loaded_baseline_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

with open(model_path_directory + 'DrivingComfortabilityPredictingModelHistory.pkl', 'rb') as history_file:
    history = pickle.load(history_file)

In [23]:
loaded_baseline_model.summary()

In [24]:
label_encoder = LabelEncoder()
encoded_labels = label_encoder.fit_transform(['Acceptable','Excellent','So and So','Uncomfortable'])

In [25]:
lateral,longitudinal = np.meshgrid(np.arange(lateral_min, lateral_max, step), np.arange(longitudinal_min, longitudinal_max, step))

In [26]:
predictions = loaded_baseline_model.predict(np.c_[lateral.ravel(), longitudinal.ravel()])
label_predictions = np.argmax(predictions, axis=1)

label_predictions_mapped = label_encoder.inverse_transform(label_predictions)

label_predictions_mapped_float = label_encoder.transform(label_predictions_mapped).astype(float)

label_predictions_reshaped = label_predictions_mapped_float.reshape(lateral.shape)

In [28]:
fig, axis = plt.subplots()
axis.set_xlim(lateral_min, lateral_max)
axis.set_ylim(longitudinal_min, longitudinal_max)
contour = axis.contourf(lateral, longitudinal, label_predictions_reshaped, cmap=custom_listed_color_map, alpha=0.8)

plt.grid(True)
plt.xlabel('Lateral acceleration')
plt.ylabel('Longitudinal acceleration')
plt.title(f'Comfort-Levels predicted by model')
plt.show()